In [1]:
## transforms

import monai
import numpy as np
from monai.transforms import (
    Compose,
    LoadImaged,
    Resample,
    NormalizeIntensityd,
    RandFlipd,
    # RandRotate10d,
    RandZoomd,
    RandAffined,
    RandGaussianNoised,   
    ToTensord,
    ScaleIntensity,
    EnsureChannelFirst,
    Resize
)

keys = ['image']

train_transforms = Compose([
    # LoadImaged(keys, image_only=True),
    ScaleIntensity(),
    EnsureChannelFirst(),
    Resize((96, 96, 96))
])
# train_transforms = Compose([
#     LoadImaged(keys, image_only=True),
#     # ToTensord(keys),
#     Resample(keys, 
#               (1.5, 1.5, 1.5)),

#     NormalizeIntensityd('image', nonzero=True, channel_wise=True),
#     RandFlipd(keys, prob=0.5, spatial_axis=0),# random by x-axis
#     RandZoomd(keys, prob=0.5, min_zoom=0.9, max_zoom=1.1),
#     RandAffined(
#         keys,
#         prob=0.5,
#         rotate_range=(0, 0, np.pi/100),
#         shear_range=(0.1, 0.1, 0.1),
#         translate_range=(5, 5, 5),
#         scale_range=(0.1, 0.1, 0.1),
#         padding_mode='border'
        
#     ),
#     RandGaussianNoised('image', prob=0.5, std=0.01),
# ])

# test_transforms = Compose([
#     LoadImaged(keys, image_only=True),
#     # ToTensord(keys),
#     Resample(keys, (1.5, 1.5, 1.5)),
#     NormalizeIntensityd('image', nonzero=True, channel_wise=True),
# ])

- dwi_adc_in_MNI_brain.nii.gz
- dwi_b0_in_MNI_brain.nii.gz
- flair_in_MNI_brain.nii.gz
- pwi_K2_in_MNI_brain.nii.gz
- pwi_rBF_in_MNI_brain.nii.gz
- pwi_ref_in_MNI_brain.nii.gz
- pwi_tMIP_in_MNI_brain.nii.gz


In [4]:
## data load
import os
import pandas as pd
from monai.data import DataLoader, ImageDataset
import glob
from sklearn.model_selection import train_test_split
import torch



df = pd.read_csv('/home/ncp/workspace/blockstorage/kyw/StudyHT_Open.csv')
mni_img_dirs = glob.glob('/home/ncp/workspace/nasr/pub66n1/topic1/ImageData/*/MNI_Space/dwi_adc_in_MNI_brain.nii.gz')
img_dirs = sorted(mni_img_dirs)
train_img_dirs = img_dirs[:120]
labels = list(map(int, list(df.loc[:119, 'GT_HTType']))) # label 

train_img_dirs, valid_img_dirs, train_labels, valid_labels = train_test_split(train_img_dirs,
                                                                              labels,
                                                                              test_size=0.3,
                                                                              stratify=labels)

In [5]:
train_dataset = ImageDataset(image_files=train_img_dirs, labels=train_labels, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=torch.cuda.is_available())
    
valid_dataset = ImageDataset(image_files=valid_img_dirs, labels=valid_labels, transform=train_transforms)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, pin_memory=torch.cuda.is_available())

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from resnet3d_pretrained import *

cfg = {
    'max_epochs' : 5,
    # 'model' : 'densenet_model',
    'opt' : 'adamw',
    'lr' : 0.001,
    'weight decay' : 1e-4,
    'label_smoothing' : 0,
    'lr_scheduler' : 'cosineannelinglr',
    'lr_warmup_epochs' : 3,
    'lr_min' : 0.0,
    'val_resize_size' : 108,
    'val_crop_size' : 96,
    'train_crop_size' : 96,
    'weight_decay' : 1e-4,
    'img_dirs' : '/home/',
    'work_dir' : '/home/',
    'lr_warmup_decay' : 0.01,
    'infererence_pretrain_dir' : '/home',
    'seed' : 66
}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet101(shortcut_type='B')
pretrained_dict = torch.load("/home/ncp/workspace/blockstorage/kyw/pretrainedmodel/resnet_101.pth", map_location=torch.device('cpu') )
model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)


<All keys matched successfully>

In [7]:
model = model.to(device)

In [12]:
from torchmetrics.classification import (
    BinaryAccuracy,
    BinaryF1Score,
    BinaryConfusionMatrix,
    BinaryAUROC
)
# loss, optimizer and scheduler

criterion = nn.CrossEntropyLoss()
if cfg['opt'] == 'adamw':
    optimizer = optim.AdamW(model.parameters(), lr=cfg['lr'],  weight_decay=cfg['weight_decay'])

# scheduler = CosineAnnealingLR(optimizer, T_max=cfg['max_epochs'], eta_min=cfg['lr_min'])

## define metrics
accuracy = BinaryAccuracy()
f1 =  BinaryF1Score()
auroc =  BinaryAUROC()

def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            all_preds.append(torch.argmax(outputs, dim=1))
            
            all_targets.append(target)

    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    acc = accuracy(all_preds, all_targets)
    f1_score = f1(all_preds, all_targets)
    # auroc_score = auroc(all_preds, all_targets)

    return acc, f1_score
                             

                              

In [15]:
from tqdm import tqdm
for epoch in range(cfg['max_epochs']):
    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluation on validation set
    val_acc, val_f1, val_auroc = evaluate(model, valid_loader)

    # Update the learing late
    # scheduler.step()
    # print(f"Epoch[{epoch+1/cfg['max_epochs']}], Loss: {loss.item():.4f}, Val Acc : {val_acc:.4f}, Val F1 : {val_f1:.4f}")
    
    

6it [03:29, 34.95s/it]


ValueError: not enough values to unpack (expected 3, got 2)